# Raw Materials Costs Forecast
### Wood Chips Costs as example

[Catalyst Paper](https://www.catalystpaper.com/) manufactures diverse paper saling it to retailers, publishers and commercial printers in North America, Latin America, the Pacific Rim and Europe. Company's production located in Canada and USA. 
Canadian facilities operate in British Columbia and purchase wood chips, which is the major raw material primarily in British Columbia. However, since wood chips is a global commodity, the price in Canadian Dollars highly correlates with exchange rate.

Becides exchange rate chips prices is subject for seasonality and general inflation.
Also, big wildfires in British Columbia in 2017 which destroyed 1.2 million hectars of forests show the significant impact wildfires on forestry products.



We use the modified data set on wood chips prices, which does not represent any actual data for any actual company and used fro demostration purposes only. CAD to USD exchange rates brought from [Bank of Canada](https://www.bankofcanada.ca/rates/exchange/)

In [2]:
# Prepare the environment
library(ggplot2)
library(data.table)
library(dplyr)

In [9]:
# Getting data
df.chips <- read.csv("Chips.csv")
df.fx <- read.csv("FX.csv")
data <- merge(df.chips, df.fx, by = c("MEDate", "YYYY", "Mmm", "Month"))
head(data)

MEDate,YYYY,Mmm,Month,Site,ProductGroup,CostElement,CAD.MT,CAD.USD
2011-01-31,2011,Jan,Jan-11,16,A1658,Chips,116.17,0.9938
2011-02-28,2011,Feb,Feb-11,16,A1658,Chips,122.78,0.9876
2011-03-31,2011,Mar,Mar-11,16,A1658,Chips,123.77,0.9766
2011-04-30,2011,Apr,Apr-11,16,A1658,Chips,124.18,0.9582
2011-05-31,2011,May,May-11,16,A1658,Chips,122.60,0.9680
2011-06-30,2011,Jun,Jun-11,16,A1658,Chips,127.85,0.9768


For the purpose of this exercise we will analyse inflation and seasonality first. After that we will add exchange rate impact to our model and finaly include wildfire data.

## Inflation

In [10]:
# Converting chips costs to USD 
data$USD.MT <- data$CAD.MT / data$CAD.USD
lrmodel1 <- lm(CAD.MT ~ YYYY, data = data)
summary(lrmodel1)


Call:
lm(formula = CAD.MT ~ YYYY, data = data)

Residuals:
     Min       1Q   Median       3Q      Max 
-16.5117  -5.0600  -0.6026   5.2440  13.1573 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -4062.4555   761.9575  -5.332 8.41e-07 ***
YYYY            2.0782     0.3783   5.493 4.33e-07 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 6.935 on 82 degrees of freedom
Multiple R-squared:  0.269,	Adjusted R-squared:  0.2601 
F-statistic: 30.17 on 1 and 82 DF,  p-value: 4.33e-07


## Seasonality

Briefly looking at the data we see the general growing trend over the course of the time. We will be recognizing that general growing trend with seasonal volatility as inflation with one exception: isolate the exchange rate and consider it as a separate factor.

In [11]:
# Shift Chip Prices in USD by one month lagging
data$USD.MT.lagged <- shift(data$USD.MT, n = 1, fill = data[1,10], type = "lag")
data$Seasonality <- data$USD.MT - data$USD.MT.lagged

# Historical seasonality ratios by month(rows) and year(columns)
M.Seasonality <- matrix(data$Seasonality, nrow = 12)
colnames(M.Seasonality) <- unique(data$YYYY)
rownames(M.Seasonality) <- unique(data$Mmm)

# Calculate average inflation per month
Avg.Seasonality <- matrix(0, nrow = 12)
for (Mnth in 1:12){
  Avg.Seasonality[Mnth] <- mean(M.Seasonality[Mnth,])
}
rownames(Avg.Seasonality) <- unique(data$Mmm)
head(Avg.Seasonality)

Jan,-6.9965044
Feb,2.0816767
Mar,2.6592342
Apr,5.3124853
May,-6.3618941
Jun,0.4534066


In [12]:
# Add Average Seasonality to the data frame
DF.Avg.Seasonality <- data.frame(Avg.Seasonality)
DF.Avg.Seasonality$Mmm <- rownames(Avg.Seasonality)
data <- merge(data, DF.Avg.Seasonality, by = "Mmm")
head(data)

Mmm,MEDate,YYYY,Month,Site,ProductGroup,CostElement,CAD.MT,CAD.USD,USD.MT,USD.MT.lagged,Seasonality,Avg.Seasonality
Apr,2011-04-30,2011,Apr-11,16,A1658,Chips,124.18,0.9582,129.59716,126.73561,2.861548,5.312485
Apr,2012-04-30,2012,Apr-12,16,A1658,Chips,126.85,0.9926,127.79569,118.78459,9.011102,5.312485
Apr,2017-04-30,2017,Apr-17,16,A1658,Chips,134.36,1.3444,99.94049,96.69107,3.249427,5.312485
Apr,2016-04-30,2016,Apr-16,16,A1658,Chips,124.68,1.2819,97.26188,90.74550,6.516376,5.312485
Apr,2013-04-30,2013,Apr-13,16,A1658,Chips,126.11,1.0187,123.79503,119.23490,4.560135,5.312485
Apr,2014-04-30,2014,Apr-14,16,A1658,Chips,130.55,1.0991,118.77900,113.05483,5.724171,5.312485


Since we have data prepared run regression for Chips Prices in CAD against inflation:

In [15]:
# Simple regression Chips price in CAD against Inflation
lrmodel2 <- lm(data$CAD.MT ~ data$Avg.Seasonality + data$YYYY)
summary(lrmodel2)


Call:
lm(formula = data$CAD.MT ~ data$Avg.Seasonality + data$YYYY)

Residuals:
     Min       1Q   Median       3Q      Max 
-15.9632  -4.3835  -0.8481   4.5355  14.3128 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)          -4062.4161   725.3772  -5.600 2.84e-07 ***
data$Avg.Seasonality     0.5601     0.1819   3.079  0.00284 ** 
data$YYYY                2.0782     0.3602   5.770 1.40e-07 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 6.602 on 81 degrees of freedom
Multiple R-squared:  0.3456,	Adjusted R-squared:  0.3294 
F-statistic: 21.39 on 2 and 81 DF,  p-value: 3.485e-08


## FX rate

We already have exchange rate as separate independant variable in separate column. Just add it to the model:

In [25]:
# Multiple regression Chips price in CAD against Inflation and FX
lrmodel3 <- lm(data$CAD.MT ~ data$Avg.Seasonality + data$YYYY + data$CAD.USD)
summary(lrmodel3)


Call:
lm(formula = data$CAD.MT ~ data$Avg.Seasonality + data$YYYY + 
    data$CAD.USD)

Residuals:
     Min       1Q   Median       3Q      Max 
-16.1698  -4.0593  -0.2585   3.9653  13.6983 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)  
(Intercept)          -2269.7288  2276.6080  -0.997   0.3231  
data$Avg.Seasonality     0.4526     0.2350   1.926   0.0592 .
data$YYYY                1.1832     1.1389   1.039   0.3033  
data$CAD.USD             8.4429    16.2333   0.520   0.6050  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 6.532 on 56 degrees of freedom
Multiple R-squared:  0.2652,	Adjusted R-squared:  0.2258 
F-statistic: 6.737 on 3 and 56 DF,  p-value: 0.0005833


## Wildfires impact on Chips Price

Wildfires sometimes have significant impact on forestry products supply and consequently influence wood chips prices. Using Fire Perimeters - Historical dataset published by the Ministry of Forests, Lands, Natural Resource Operations and Rural Development - BC Wildfire Services we apply area of wildfires by month occur in British Columbia.
We import this data as it came from the source and need to work on it before add to our data frame.

In [17]:
df.wildfires <- read.csv("H_FIRE_PLY.csv")                # Dataset as it is provided by the source
head(df.wildfires)

SOURCE,VERSION_NO,FIRE_CAUSE,SIZE_HA,FIRE_YEAR,TRACK_DATE,LOAD_DATE,FIRE_NO,FIRE_DATE,FIRELABEL,FCODE,METHOD,OBJECTID,SHAPE,X,Y
linens,NA,Person,1510.4,1932,NA,2.007052e+13,106,1.932072e+13,1932-106,JA70003000,digitised,128146,NA,NA,NA
linens,NA,Lightning,1459.4,1936,NA,2.007052e+13,122,1.936063e+13,1936-122,JA70003000,digitised,128147,NA,NA,NA
linens,NA,Person,6.5,1935,NA,2.007052e+13,174,1.935091e+13,1935-174,JA70003000,digitised,128148,NA,NA,NA
linens,NA,Person,15.0,1931,NA,2.007052e+13,18,1.931043e+13,1931-18,JA70003000,digitised,128149,NA,NA,NA
linens,NA,Person,73.6,1931,NA,2.007052e+13,19,1.931043e+13,1931-19,JA70003000,digitised,128150,NA,NA,NA
linens,NA,Person,146.8,1931,NA,2.007052e+13,20,1.931043e+13,1931-20,JA70003000,digitised,128151,NA,NA,NA


In [18]:
df.wildfires <- df.wildfires %>% filter(as.numeric(substr(df.wildfires$FIRELABEL, 1, 4)) >= 2011 )
df.wildfires

SOURCE,VERSION_NO,FIRE_CAUSE,SIZE_HA,FIRE_YEAR,TRACK_DATE,LOAD_DATE,FIRE_NO,FIRE_DATE,FIRELABEL,FCODE,METHOD,OBJECTID,SHAPE,X,Y
OZIX,2012040701,Person,10.6,2012,2.012041e+13,2.012041e+13,K60009,2.012041e+13,2012-K60009,JA70003000,GPS-TRACK,147949,NA,NA,NA
OZIX,2012041701,Person,1.4,2012,2.012042e+13,2.012042e+13,C50006,2.012041e+13,2012-C50006,JA70003000,GPS-TRACK,147950,NA,NA,NA
OZIX,2012042202,Person,6.6,2012,2.012042e+13,2.012042e+13,K60032,2.012042e+13,2012-K60032,JA70003000,GPS-TRACK,147951,NA,NA,NA
OZIX,2011102701,Person,21.3,2011,2.011112e+13,2.011112e+13,G40141,2.011102e+13,2011-G40141,JA70003000,GPS-TRACK,147723,NA,NA,NA
OZIX,2011102801,Person,106.7,2011,2.011112e+13,2.011112e+13,G40145,2.011102e+13,2011-G40145,JA70003000,GPS-TRACK,147724,NA,NA,NA
OZIX,2011103101,Person,38.2,2011,2.011112e+13,2.011112e+13,G40150,2.011102e+13,2011-G40150,JA70003000,GPS-TRACK,147725,NA,NA,NA
OZIX,2011103101,Person,14.6,2011,2.011112e+13,2.011112e+13,G40151,2.011102e+13,2011-G40151,JA70003000,GPS-TRACK,147726,NA,NA,NA
OZIX,2011110201,Person,13.8,2011,2.011112e+13,2.011112e+13,G40154,2.011103e+13,2011-G40154,JA70003000,GPS-TRACK,147727,NA,NA,NA
OZIX,2011110202,Person,103.0,2011,2.011113e+13,2.011113e+13,C10098,2.011110e+13,2011-C10098,JA70003000,GPS-TRACK,147728,NA,NA,NA
OZIX,2012031001,Person,3.3,2011,2.012032e+13,2.012032e+13,K20781,2.012031e+13,2011-K20781,JA70003000,GPS-TRACK,147729,NA,NA,NA


In [19]:

df.wildfires$YYYY <- substr(df.wildfires$FIRELABEL, 1, 4) # Extracting calendar year of incident
df.wildfires$MM <- substr(df.wildfires$FIRE_DATE, 6,7)    # Extracting calendar month of incident

# Mapping short month name to month index MM
MM <- c("01","02","03","04","05","06","07","08","09","10","11","12")
Mmm <- c("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec")
df.Mmm <- data.frame(MM, Mmm)

df.wildfires <- merge(df.wildfires, df.Mmm, by = "MM")
df.wildfires$Month <- paste0(df.wildfires$Mmm, "-", substr(df.wildfires$YYYY,3,4))

# Grouping Fire Size by Month
df.wildfires <- df.wildfires %>% group_by(Month) %>% summarize (Fire.Area = sum(SIZE_HA))
head(df.wildfires)

Month,Fire.Area
Apr-11,82.4
Apr-12,447.4
Apr-13,2917.7
Apr-14,674.4
Apr-15,874.9
Apr-16,112975.1


Now we can merge wildfire data with our main data frame using columns YYYY and Mmm as a key.

In [20]:
data <- merge(data, df.wildfires, by = "Month")
head(data)

Month,Mmm,MEDate,YYYY,Site,ProductGroup,CostElement,CAD.MT,CAD.USD,USD.MT,USD.MT.lagged,Seasonality,Avg.Seasonality,Fire.Area
Apr-11,Apr,2011-04-30,2011,16,A1658,Chips,124.18,0.9582,129.59716,126.7356,2.861548,5.312485,82.4
Apr-12,Apr,2012-04-30,2012,16,A1658,Chips,126.85,0.9926,127.79569,118.7846,9.011102,5.312485,447.4
Apr-13,Apr,2013-04-30,2013,16,A1658,Chips,126.11,1.0187,123.79503,119.2349,4.560135,5.312485,2917.7
Apr-14,Apr,2014-04-30,2014,16,A1658,Chips,130.55,1.0991,118.77900,113.0548,5.724171,5.312485,674.4
Apr-15,Apr,2015-04-30,2015,16,A1658,Chips,129.88,1.2331,105.32804,100.0634,5.264639,5.312485,874.9
Apr-16,Apr,2016-04-30,2016,16,A1658,Chips,124.68,1.2819,97.26188,90.7455,6.516376,5.312485,112975.1


Calculate regression for Chips Costs against 

In [21]:
lrmodel4 <- lm(data$CAD.MT ~ data$Avg.Seasonality + data$YYYY + data$CAD.USD + data$Fire.Area)
summary(lrmodel4)


Call:
lm(formula = data$CAD.MT ~ data$Avg.Seasonality + data$YYYY + 
    data$CAD.USD + data$Fire.Area)

Residuals:
     Min       1Q   Median       3Q      Max 
-15.9602  -3.8115   0.1661   4.1129  13.6696 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)  
(Intercept)          -1.028e+03  2.330e+03  -0.441   0.6608  
data$Avg.Seasonality  4.635e-01  2.302e-01   2.014   0.0490 *
data$YYYY             5.627e-01  1.165e+00   0.483   0.6311  
data$CAD.USD          1.511e+01  1.631e+01   0.927   0.3581  
data$Fire.Area        1.065e-05  5.788e-06   1.839   0.0713 .
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 6.397 on 55 degrees of freedom
Multiple R-squared:  0.3078,	Adjusted R-squared:  0.2574 
F-statistic: 6.113 on 4 and 55 DF,  p-value: 0.0003828
